In [1]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from Dataset.Economy_Dataset import Economy_Dataset
from Dataset.RNN_Transformer_Dataset import RNN_Transformer_Dataset

from Model.SelfAttention import Self_attention, Attention_LSTM
from Model.LSTM import LSTM

import matplotlib.pyplot as plt

In [2]:
# 데이터 다운 & 전처리

transaction_df = pd.read_excel('../데이터/transaction/transaction_final.xlsx',index_col=0)
economy_df = pd.read_excel('../데이터/economy/economy_all.xlsx')['국고채금리']

In [3]:
print(len(transaction_df))
print(len(economy_df))

246685
204


In [62]:
# len_df = len(transaction_df)
# train_len = int(len_df *0.7)
# val_len = int(len_df * 0.3)

batch_size = 4
train_dataset = RNN_Transformer_Dataset(transaction_df, economy_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True)

# val_dataset = RNN_Transformer_Dataset(df[train_len:train_len+val_len])
# val_loader = DataLoader(val_dataset, batch_size=batch_size)

c:\Users\hkyoo\OneDrive\바탕 화면\SCI\코드\utils.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['계약년월'] = pd.to_datetime(df['계약년월'])


In [63]:
for x,y,z,w in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  print("Z 크기 : {}".format(z.shape))
  print("W 크기 : {}".format(w.shape))


X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 21, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 21, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 21, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 21, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 21, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 21, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 21, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5])
Z 크기 : torch.Size([4, 21, 1])
W 크기 : torch.Size([4, 1, 1])
X 크기 : torch.Size([4, 21, 5])
Y 크기 : torch.Size([4, 1, 5

In [66]:
max_size = 158      #################
window_size = 5
hidden_size =256

In [80]:
# gpu 연결
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

# checkpoint 받기
eco_lstm = LSTM(window_size,256).to(device)
eco_lstm.load_state_dict(torch.load("../데이터/checkpoint/best_rnn_economy_model.pth"))

tran_lstm = LSTM(window_size,256).to(device)
tran_lstm.load_state_dict(torch.load("../데이터/checkpoint/best_rnn_transaction_model.pth"))

model = Self_attention(max_size, hidden_size, 1).to(device)

dummy = torch.rand((batch_size,max_size,hidden_size)).to(device)
model(dummy)

cpu is available


tensor([[[0.1152],
         [0.1108],
         [0.1238],
         [0.1191],
         [0.0859],
         [0.0959],
         [0.1435],
         [0.0741],
         [0.0929],
         [0.1178],
         [0.1342],
         [0.1582],
         [0.1188],
         [0.1180],
         [0.0951],
         [0.0798],
         [0.1131],
         [0.1350],
         [0.1124],
         [0.0826],
         [0.1237]],

        [[0.1910],
         [0.2394],
         [0.1639],
         [0.1901],
         [0.2245],
         [0.2106],
         [0.1884],
         [0.2095],
         [0.1885],
         [0.1736],
         [0.1966],
         [0.1723],
         [0.1658],
         [0.1721],
         [0.2082],
         [0.2124],
         [0.1709],
         [0.2171],
         [0.1902],
         [0.2017],
         [0.1490]],

        [[0.1882],
         [0.1824],
         [0.1799],
         [0.1920],
         [0.1936],
         [0.1914],
         [0.1806],
         [0.2211],
         [0.1831],
         [0.1758],
        

In [82]:
criterion = nn.MSELoss()

lr = 1e-3
num_epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### 학습하기

In [69]:
train_n = len(train_dataset)
# val_n = len(val_dataset)

# val_loss_list = []
best_train_loss = float('inf')
stop_counter = 0
stop_criteria = 3
max_len = max_size

for epoch in range(num_epochs):
    print('epoch : {}'.format(epoch))
    running_loss = 0.0
    num = 0
    print("train")
    model.train()
    for tran_inputs, eco_inputs, tran_labels, eco_label in train_loader:  # dataloader 함수에 def __len__에 return 값만큼 반복함
        tran_inputs = tran_inputs.to(device) 
        eco_inputs = eco_inputs.to(device)
        tran_labels = tran_labels.to(device)  # batch_size*max_size*1
        eco_label = eco_label.to(device) 
        
        # transaction hidden_state + economy hidden_state
        # batch 단위로 한번에 더함
        # 콜금리 사용
        eco_tensor = eco_lstm(eco_inputs)[1][0].squeeze().unsqueeze(1).to(device)   # batch * hidden_state
        eco_tensor = eco_tensor.repeat(1,max_len,1)
        tran_tensor = torch.zeros((batch_size,max_len,hidden_size)).to(device)
        for i in range(max_len):
            tran_tensor[:,i,:] = tran_lstm(tran_inputs)[1][0].squeeze()
            
        emb_vector = tran_tensor + eco_tensor
        
        # 부동산 데이터가 0인 행은 embedding_vector도 0으로 전환  
        iszero_vector = torch.where(tran_labels.squeeze() == 0, tran_labels.squeeze(), 1.0)
           
        # 차원에 맞게 변경 후 계산 
        emb_vector = emb_vector.mul(iszero_vector.unsqueeze(2).repeat(1,1,hidden_size))
        
        #####################################
        # transformer 적용
        # loss 값은 부동산 가격이 0이 아닌 것에만 적용해서 학습
        out = model(emb_vector) # batch_size*max_size*1
        
        # out 값중 필요 없는 값은 0으로 변경
        out = out.mul(iszero_vector.unsqueeze(2))   
        
        labels = tran_labels.mul(iszero_vector.unsqueeze(2))
        
        # x값이 0이면 y값도 0인 상태!!!에서 아래 진행
        loss_all = 0
        for i in range(out.shape[1]):
            loss = criterion(out[:,i,:], labels[:,i,:])
            loss_all += loss
        loss_all = loss_all/iszero_vector.sum()   # 평균치만큼 업데이트!!!!!!!!
        
        mean_loss = loss_all/out.shape[1]
        optimizer.zero_grad() #
        mean_loss.backward() # loss가 최소가 되게하는
        optimizer.step() # 어떤 방법으로 learning rate를 움직일거이냐
        running_loss += loss_all.item() # 한 배치의 loss 더해주고,

    if running_loss < best_train_loss:
        best_train_loss = running_loss
        torch.save(model.state_dict(), '../데이터/checkpoint/best_rnn_transformer_model.pth')

    print(running_loss/train_n)

    # with torch.no_grad():
    #         print("Calculating validation results...")
    #         model.eval()
    #         val_loss = 0
    #         for inputs, labels in val_loader:
    #             price, economy = inputs
    #             price = price.to(device) # batch_size*max_size*256  (max_size = 최대 단지 개수)
    #             economy = economy.to(device)
    #             labels = labels.to(device) # batch_size*max_size*1
                
    #             # labels 가 0이 아니면 lstm 넣기
    #             embed = torch.zeros((batch_size,max_size,hidden_size))
    #             for i in range(labels.shape[0]):
    #                 for j in range(labels.shape[1]):
    #                     if labels[i][j] != 0:
    #                         embed[i][j] = lstm_price(price[i][j]) 
                            
    #             outs = model(inputs)

    #             loss_all = 0
    #             for i in range(out.shape[1]):
    #                 loss = criterion(out[:,i,:], labels[:,i,:])
    #                 loss_all += loss

    #         avg_val_loss = val_loss/val_n/out.shape[1]
    #         val_loss_list.append(avg_val_loss)

    #         if avg_val_loss < best_val_loss:
    #             best_val_loss = avg_val_loss
    #             stop_counter = 0            
    #             torch.save(model.state_dict(), 'Checkpoint/economic_best_model.pth')
    #         else:
    #             stop_counter += 1

    #         print("val")
    #         print(
    #             f"loss: {avg_val_loss:4.8} || "
    #         )

    #         if stop_counter >= stop_criteria:
    #             print('Early stopping triggered.')
    #             break
    
    print('======================================================')

# plt.figure(figsize=(10, 5))
# plt.plot(val_loss_list, label='Validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

epoch : 0
train
nan
epoch : 1
train
nan
epoch : 2
train
nan
epoch : 3
train
nan
epoch : 4
train
nan
epoch : 5
train
nan
epoch : 6
train
nan
epoch : 7
train
nan
epoch : 8
train
nan
epoch : 9
train
nan
epoch : 10
train
nan
epoch : 11
train
nan
epoch : 12
train
nan
epoch : 13
train
nan
epoch : 14
train
nan
epoch : 15
train
nan
epoch : 16
train
nan
epoch : 17
train
nan
epoch : 18
train
nan
epoch : 19
train


KeyboardInterrupt: 